In [ ]:
# installing necessary packages
pip install statsbombpy
pip install plotly

In [2]:
# importing libraries
import json
from pandas import json_normalize
import random
from plotly import graph_objects, offline
import os
from collections import defaultdict
from statsbombpy import sb


class Field ():

    def __init__(self):
        figure = graph_objects.Figure()

        figure.update_layout(width=900*1.388, height=900, autosize=False, plot_bgcolor="white")
        figure.update_xaxes(range=[-0.03, 1.03])
        figure.update_yaxes(range=[-0.03, 1.03])

        self.figure = figure

        self.__draw_full()

    def add_title(self, main_title: str):
        """
        Adds a title to the pitch visualization
        """
        self.figure.update_layout(title={'text': main_title, 'x': 0.48, 'y': 0.91,
                                         'xanchor': 'center', 'yanchor': 'top'})

    def save(self, name: str):
        """Saves PNG and html image"""
        offline.plot(self.figure, filename=f"{name}", auto_open=False)

    def __draw_full(self):
        # Contour
        self.figure.add_shape(type="rect", x0=0, y0=0, x1=1, y1=1)
        # Half way line
        self.figure.add_shape(type="line", x0=0.5, y0=0, x1=0.5, y1=1)
        circle_radius = 0.0915
        self.figure.add_shape(type="circle", x0=0.5-circle_radius*0.72, y0=0.5 -
                              circle_radius, x1=0.5+circle_radius*0.72, y1=0.5+circle_radius)
        # Defensive goal area
        self.figure.add_shape(type="rect", x0=0, y0=0.211, x1=0.170, y1=0.789)
        self.figure.add_shape(type="rect", x0=0, y0=0.368, x1=0.058, y1=0.632)
        self.figure.add_shape(type="rect", x0=-0.01, y0=0.447, x1=0.0, y1=0.553)
        # Attacking goal area
        self.figure.add_shape(type="rect", x0=0.83, y0=0.211, x1=1.0, y1=0.789)
        self.figure.add_shape(type="rect", x0=0.942, y0=0.368, x1=1.0, y1=0.632)
        self.figure.add_shape(type="rect", x0=1.0, y0=0.447, x1=1.01, y1=0.553)


def add_possession_chain_sb(figure, chain, team_name: str, opacity: int = 1.0):
    this_team_color = "#1F77B4"
    other_team_color = "#FF7F0E"

    chain_x = []
    chain_y = []
    text = []
    colors = []
    outcomes = []
    times = []
    counter = 0
    try:
        for idx, row in chain.iterrows():
            event_name = row['type_name']
            event_loc = row['location']

            # Transform  coordinates
            x = round((event_loc[0] * (100 / 120)) / 100, 3)
            y = round(((80 - event_loc[1]) * (100 / 80)) / 100, 3)  # Reverse it because opta pitch is 0,100 and statsbomb 100,0

            color = this_team_color

            # slightly shift events that has the same coodinates
            if x in chain_x and y in chain_y:
                x = x + random.choice([-0.001, 0.001])
                y = y + random.choice([-0.001, 0.001])

            if row['possession_team_id'] != row['team_id']:
                color = other_team_color
                x = round(1.000 - x, 3)
                y = round(1.000 - y, 3)

            chain_x.append(x)
            chain_y.append(y)
            if row['possession_team_id'] == row['team_id']:
                text.append(f"{counter}.{event_name}")
            else:
                text.append(f"{counter}.{event_name}")

            colors.append(color)
            times.append(f"{row['minute']}{row['second']}")
            outcomes.append('circle')
            counter += 1

    except Exception as err:
        print(err)

    line_color = "#7F7F7F"

    figure.add_trace(
        {'mode': 'markers+lines+text',
         'textposition': 'top center',
         'type': 'scatter',
         'x': chain_x,
         'y': chain_y,
         'hovertemplate': "Time: %{hovertext}<br>Event: %{text}",
         'text': text,
         'opacity': opacity,
         'hovertext': times,
         'marker': {'color': colors, 'size': 8, 'symbol': outcomes},
         'line': {'color': line_color, 'dash': 'solid'},
         'name': "{} {}:{} ".format(chain.iloc[0]['minute'], chain.iloc[0]['second'],team_name)
         }
    )

# MAIN METHOD


# Load in the data 
# Load in match events

import json
with open(r'C:\Users\danys\Documents\Soccer_analytics\soccer_data\open-data-master\data\events\7548.json') as data_file:
    data = json.load(data_file)



match_events = json_normalize(data, sep="_") # Tranform data to DataFrmae

relevand_possessions = match_events[(match_events['type_id'] == 16)]['possession'].unique()  # all uniques possessions with shot

# Create Pitch
ground = Field()
ground.add_title('Possession chains')

for possession in relevand_possessions:

    df = match_events[match_events['possession'] == possession] # get all events of the possession

    # add each event
    add_possession_chain_sb(ground.figure, df, team_name= df.iloc[0]['team_name'])

#Look in the file below for the visulasiation of chains
ground.save('possessions.html')
print('completed')

'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
completed


In [53]:
# for the future sorting, we need to separate events
season_ids = sb.competitions().season_id.unique()
competition_ids = sb.competitions().competition_id.unique()
competition_names = sb.competitions().competition_name.unique()
country_names = sb.competitions().country_name.unique()

In [58]:
print('Competition ids: \n', competition_ids)
print('Season ids: \n', season_ids)
print('Competition names: \n', competition_names)
print('Country names: \n', country_names)

Competition ids: 
 [  16   37   43 1238   11   49    2   55   53   72]
Season ids: 
 [  4   1   2  27  26  25  24  23  22  21  41  39  37  44  76  90  42   3
 108  40  38  43 106  30]
Competition names: 
 ['Champions League' "FA Women's Super League" 'FIFA World Cup'
 'Indian Super league' 'La Liga' 'NWSL' 'Premier League' 'UEFA Euro'
 "UEFA Women's Euro" "Women's World Cup"]
Country names: 
 ['Europe' 'England' 'International' 'India' 'Spain'
 'United States of America']
